In [1]:
import numpy as np

from live_strat import RSIRisingFalling

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="1h",
    candles_to_dl=3000,
)

In [3]:
short_strat = RSIRisingFalling(
    long_short="short",
    rsi_length=np.array([14]),
    rsi_is_above=np.arange(50, 76, 5),
)
short_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
short_strat.plot_signals(candles=candles)

In [4]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(35, 61, 5),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
long_strat.plot_signals(candles=candles)

In [5]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1, 0.25, 0.5]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [6]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 48
Total combinations of settings to test: 288
Total candles: 3,000
Total candles to test: 864,000


In [7]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
263,5,23,37.0,23,14,497.350,62.162,0.937,419.363,4973.502,5973.502
211,4,19,54.0,28,26,1070.079,51.852,0.932,1728.211,10700.790,11700.790
207,4,15,39.0,24,15,530.912,61.538,0.932,679.902,5309.123,6309.123
260,5,20,74.0,41,33,304.502,55.405,0.931,920.451,3045.018,4045.018
255,5,15,41.0,24,17,515.124,58.537,0.929,498.123,5151.237,6151.237
262,5,22,63.0,31,32,305.437,49.206,0.925,765.999,3054.371,4054.371
279,5,39,41.0,24,17,534.563,58.537,0.924,607.965,5345.629,6345.629
209,4,17,55.0,28,27,1028.531,50.909,0.923,1915.412,10285.309,11285.309
252,5,12,77.0,41,36,291.280,53.247,0.922,1058.814,2912.804,3912.804
254,5,14,66.0,31,35,340.032,46.970,0.921,1003.435,3400.320,4400.320


In [13]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    dos_index=23,
    ind_set_index=5,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = 0.005,
    sl_based_on_lookback = 50,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct = 0.02,
)
IndicatorSettingsArrays(
    rsi_is_above = nan,
    rsi_is_below = 60.0,
    rsi_length = 14,
)


In [9]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,4,19,50,1692896400000,2023-08-24 17:00:00,EntryFilled,1000.000,956.273,2691.846,43.727,...,2735.573,26103.9,NaN,0.105,2735.573,NaN,1.0,25848.8,2.3,26700.1
1,4,19,50,1692896400000,2023-08-24 17:00:00,StopLossFilled,969.941,969.941,NaN,NaN,...,NaN,NaN,25848.8,NaN,NaN,-30.059,NaN,NaN,NaN,NaN
2,4,19,53,1692907200000,2023-08-24 20:00:00,EntryFilled,969.941,930.070,2148.261,39.871,...,2188.132,26027.0,NaN,0.084,2188.132,NaN,1.2,25713.1,2.7,26740.5
3,4,19,55,1692914400000,2023-08-24 22:00:00,EntryFilled,969.941,852.914,6120.724,117.027,...,1901.358,26079.8,NaN,0.157,4089.490,NaN,1.3,25713.1,2.9,26806.1
4,4,19,59,1692928800000,2023-08-25 02:00:00,EntryFilled,969.941,738.503,11711.963,231.438,...,1733.187,26137.6,NaN,0.223,5822.677,NaN,1.4,25713.1,3.1,26867.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,4,19,2983,1703455200000,2023-12-24 22:00:00,StopLossFilled,11700.790,11700.790,NaN,NaN,...,NaN,NaN,43173.6,NaN,NaN,-1588.537,NaN,NaN,NaN,NaN
271,4,19,2985,1703462400000,2023-12-25 00:00:00,EntryFilled,11700.790,11243.137,21021.346,457.653,...,21478.999,43008.1,NaN,0.499,21478.999,NaN,1.5,42356.5,3.4,44452.9
272,4,19,2987,1703469600000,2023-12-25 02:00:00,EntryFilled,11700.790,10346.299,60845.338,1354.491,...,19699.484,43072.7,NaN,0.956,41178.483,NaN,1.6,42356.5,3.5,44535.3
273,4,19,2991,1703484000000,2023-12-25 06:00:00,EntryFilled,11700.790,9028.446,115594.090,2672.344,...,16242.613,43265.1,NaN,1.331,57421.096,NaN,1.7,42356.5,3.8,44700.1
